### Library

In [70]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np

### Obtain

#### Inpatient Prospective Payment System (IPPS) Provider Summary for the Top 100 Diagnosis-Related Groups (DRG) - FY2011

https://data.cms.gov/Medicare-Inpatient/Inpatient-Prospective-Payment-System-IPPS-Provider/97k6-zzx3

In [2]:
data = pd.read_csv('Inpatient_Prospective_Payment_System__IPPS__Provider_Summary_for_the_Top_100_Diagnosis-Related_Groups__DRG__-_FY2011.csv')

In [3]:
data['DRG_id'] = data['DRG Definition'].apply(lambda x: x[:3])

In [4]:
data['DRG_label'] = data['DRG Definition'].apply(lambda x: x.split(' - ')[1].split(' W')[0])

In [5]:
data['without_ccmcc'] = data['DRG Definition'].apply(lambda x: 1 if ' W/O ' in x else 0)
data['with_mcc'] = data['DRG Definition'].apply(lambda x: 1 if ' W MCC' in x else 0)
data['with_cc'] = data['DRG Definition'].apply(lambda x: 1 if ' W CC'  in x else 0)
data['with_ccmcc'] = data['DRG Definition'].apply(lambda x: 1 if ' W CC/MCC'  in x else 0)
data.with_cc = data.with_cc - data.with_ccmcc

In [6]:
dict_transfer = {}
for i in data.DRG_id.unique():
    dict_transfer.update({i:data.loc[data.DRG_id == i][' Average Total Payments '].max()})


In [7]:
data['ratio_to_max_payment'] = data[' Average Total Payments ']/ data.DRG_id.apply(lambda x: dict_transfer[x])

In [8]:
dict_transfer = {}
for i in data.DRG_id.unique():
    dict_transfer.update({i:data.loc[data.DRG_id == i][' Total Discharges '].max()})


In [9]:
data['ratio_to_max_discharge'] = data[' Total Discharges ']/ data.DRG_id.apply(lambda x: dict_transfer[x])

In [10]:
transfer_dict = {}
for i in data.columns:
    j = i.replace('!!','_')
    j = j.replace(' ', '_')
    transfer_dict.update({i:j})
data = data.rename(columns = transfer_dict)


In [11]:
data['CountyState'] = data.county_name + data.Provider_State
data['CountyState'] = data['CountyState'].str.lower()
data['CountyState'] = data['CountyState'].apply(lambda x: x.replace('saint','st.'))

### American Community Study

#### Poverty

#### Poverty
https://data.census.gov/cedsci/table?q=poverty&g=0100000US.050000&table=S1701&tid=ACSST1Y2010.S1701&t=Income%20and%20Poverty%3APoverty&lastDisplayedRow=28&vintage=2018&hidePreview=true&moe=false&y=2010&d=ACS%201-Year%20Estimates%20Subject%20Tables

In [113]:
acs_income2_data = pd.read_csv('ACSDP5Y2012.DP03_data_with_overlays_2019-12-31T163946.csv', skiprows=1)
acs_income2_data['Estimate!!EMPLOYMENT STATUS!!Percent Unemployed'].unique()

C:\Users\FlatIron_User\.conda\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (60,61,68,69,274,275,290,291,404,405,440,441,445,446,448,449,452,453,480,481,484,485,492,493,496,497,500,501,504,505,516,517,520,521,524,525,528,529) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array(['(X)'], dtype=object)

In [115]:
acs_income2_data = pd.read_csv('ACSDP5Y2012.DP03_data_with_overlays_2019-12-31T163946.csv', skiprows=1)
print(len(acs_income2_data.columns))

#Removing error columns
for i in acs_income2_data.columns:
    if 'Error' in i:
        acs_income2_data = acs_income2_data.drop(columns = [i])
print(len(acs_income2_data.columns))
        
#converting 'N' to zeros
acs_income2_data = acs_income2_data.apply(lambda x: x.replace('N', 0.0))
print(len(acs_income2_data.columns))


#Remove object columns
for i in acs_income2_data.columns:
    if (acs_income2_data[i].astype(str) == '(X)').sum():
        acs_income2_data = acs_income2_data.drop(columns = [i])   
print(len(acs_income2_data.columns))



C:\Users\FlatIron_User\.conda\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (60,61,68,69,274,275,290,291,404,405,440,441,445,446,448,449,452,453,480,481,484,485,492,493,496,497,500,501,504,505,516,517,520,521,524,525,528,529) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


550
276
276
240


In [124]:
city = acs_income2_data.loc[acs_income2_data['Geographic Area Name'].apply(lambda x: 'city' in x)].index
city = acs_income2_data.iloc[city]['Geographic Area Name'].apply(lambda x: x.split(' city')[0]).values

index_ = []
for i in city:
    print(i)
    for j in acs_income2_data.loc[acs_income2_data['Geographic Area Name'].apply(lambda x: i in x)].index:
        index_.append(j)
len(index_)
        


Baltimore
St. Louis
Manassas Park
Martinsville
Newport News
Norfolk
Norton
Petersburg
Poquoson
Portsmouth
Radford
Richmond
Roanoke
Salem
Staunton
Suffolk
Virginia Beach
Waynesboro
Williamsburg
Winchester
Alexandria
Bedford
Bristol
Buena Vista
Charlottesville
Chesapeake
Colonial Heights
Covington
Danville
Emporia
Fairfax
Falls Church
Franklin
Fredericksburg
Galax
Hampton
Harrisonburg
Hopewell
Lexington
Lynchburg
Manassas


95

In [102]:
for i in acs_income2_data.columns:
    print(i)

id
Geographic Area Name
Total!!Estimate!!Population for whom poverty status is determined
Total!!Estimate!!AGE!!Under 18 years
Total!!Estimate!!AGE!!Under 18 years!!Related children under 18 years
Total!!Estimate!!AGE!!18 to 64 years
Total!!Estimate!!AGE!!65 years and over
Total!!Estimate!!SEX!!Male
Total!!Estimate!!SEX!!Female
Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!One race
Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!White
Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Black or African American
Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!American Indian and Alaska Native
Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Asian
Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Native Hawaiian and Other Pacific Islander
Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Some other race
Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races
Total!!Estimate!!Hispanic or

In [96]:
a = acs_income2_data.apply(lambda x: x.replace('N', 0.0))



In [12]:
acs_data = pd.read_csv('ACSDP5Y2017.DP05_data_with_overlays_2019-12-16T161924.csv',skiprows = 1)
acs_income_data = pd.read_csv('ACSST5Y2017.S1701_data_with_overlays_2019-12-17T171021.csv', skiprows = 1)
acs_income2_data = pd.read_csv('ACSST1Y2012.S1901_data_with_overlays_2019-12-21T221020.csv', skiprows=1)

C:\Users\FlatIron_User\.conda\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (12,13,24,25,30,31,126,127,324,325,330,331,336,337,342,343) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
for i in acs_data.columns:
    if 'Error' in i:
        acs_data = acs_data.drop(columns = [i])

for i in acs_income_data.columns:
    if 'Error' in i:
        acs_income_data = acs_income_data.drop(columns = [i])
        
for i in acs_income2_data.columns:
    if 'Error' in i:
        acs_income2_data = acs_income2_data.drop(columns = [i])
        
for i in acs_income2_data.columns:
    if (acs_income2_data[i].astype(str) == '(X)').sum():
        acs_income2_data = acs_income2_data.drop(columns = [i])    

In [14]:
acs_data['County'] = acs_data['Geographic Area Name'].apply(lambda x: x.split(',')[0][:-7])
acs_data['State'] = acs_data['Geographic Area Name'].apply(lambda x: x.split(',')[1])

In [15]:
state_abb ={"Alabama":"AL",
"Alaska":"AK",
"Arizona":"AZ",
"Arkansas":"AR",
"California":"CA",
"Colorado":"CO",
"Connecticut":"CT",
"Delaware":"DE",
"Florida":"FL",
"Georgia":"GA",
"Hawaii":"HI",
"Idaho":"ID",
"Illinois":"IL",
"Indiana":"IN",
"Iowa":"IA",
"Kansas":"KS",
"Kentucky":"KY",
"Louisiana":"LA",
"Maine":"ME",
"Maryland":"MD",
"Massachusetts":"MA",
"Michigan":"MI",
"Minnesota":"MN",
"Mississippi":"MS",
"Missouri":"MO",
"Montana":"MT",
"Nebraska":"NE",
"Nevada":"NV",
"New Hampshire":"NH",
"New Jersey":"NJ",
"New Mexico":"NM",
"New York":"NY",
"North Carolina":"NC",
"North Dakota":"ND",
"Ohio":"OH",
"Oklahoma":"OK",
"Oregon":"OR",
"Pennsylvania":"PA",
"Rhode Island":"RI",
"South Carolina":"SC",
"South Dakota":"SD",
"Tennessee":"TN",
"Texas":"TX",
"Utah":"UT",
"Vermont":"VT",
"Virginia":"VA",
"Washington":"WA",
"West Virginia":"WV",
"Wisconsin":"WI",
"Wyoming":"WY",
"Puerto Rico":"PR",
           "District of Columbia":"DC"}



In [16]:
acs_data.State = acs_data.State.apply(lambda x: state_abb[x[1:]])

In [17]:
acs_data['CountyState'] = acs_data.County + acs_data.State
acs_data['CountyState'] = acs_data['CountyState'].str.lower()


In [18]:
acs_income_data = acs_income_data.drop(columns='Geographic Area Name')
acs_income2_data = acs_income2_data.drop(columns='Geographic Area Name')
acs_income_data = acs_income_data.merge(acs_income2_data, how='left', on= 'id')
acs_data = acs_data.merge(acs_income_data, how='left', on= 'id')

In [19]:
transfer_dict = {}
for i in acs_data.columns:
    j = i.replace('!!',' ')
    j = j.replace(' ','_')
    j = j.replace('Estimate_','')
    transfer_dict.update({i:j})

acs_data = acs_data.rename(columns=transfer_dict)

In [20]:
hold_df = acs_data[['County','State','CountyState']]

for i in acs_data.columns:
    if acs_data[i].dtype == object:
        acs_data = acs_data.drop(columns=i)
        
acs_data[hold_df.columns] = hold_df

In [21]:
acs_data = acs_data.fillna(0)

for i in acs_data.columns:
    if acs_data[i].isna().sum() > 0:
        print(i, acs_data[i].isna().sum())

In [50]:
data.DRG_label.value_counts().unique()

array([8174, 8154, 8104, 6646, 6256, 5734, 5696, 5660, 4941, 4651, 4553,
       4520, 4059, 4007, 3847, 3752, 3682, 3456, 3274, 3217, 3158, 3156,
       2638, 2413, 2400, 2288, 2278, 2230, 2225, 2096, 1962, 1830, 1820,
       1801, 1794, 1793, 1593, 1545, 1396, 1376, 1332, 1249, 1233, 1201,
       1142, 1129, 1079, 1054,  988,  984,  979,  975,  955,  906,  846,
        613], dtype=int64)

### Merging 

In [35]:
df = data.merge(acs_data, how='inner', left_on=['county_name', 'Provider_State'], right_on=['County', ])
len(df), len(acs_data), len(data)

(165908, 3220, 163065)

In [23]:
drop_columns = ['SEX_AND_AGE_Total_population_Male',
                'Percent_SEX_AND_AGE_Total_population',
'SEX_AND_AGE_Total_population_Female',
'SEX_AND_AGE_Total_population_Under_5_years',
'SEX_AND_AGE_Total_population_5_to_9_years',
'SEX_AND_AGE_Total_population_10_to_14_years',
'SEX_AND_AGE_Total_population_15_to_19_years',
'SEX_AND_AGE_Total_population_20_to_24_years',
'SEX_AND_AGE_Total_population_25_to_34_years',
'SEX_AND_AGE_Total_population_35_to_44_years',
'SEX_AND_AGE_Total_population_45_to_54_years',
'SEX_AND_AGE_Total_population_55_to_59_years',
'SEX_AND_AGE_Total_population_60_to_64_years',
'SEX_AND_AGE_Total_population_65_to_74_years',
'SEX_AND_AGE_Total_population_75_to_84_years',
'SEX_AND_AGE_Total_population_85_years_and_over',
'SEX_AND_AGE_Total_population_Under_18_years',
'SEX_AND_AGE_Total_population_16_years_and_over',
'SEX_AND_AGE_Total_population_18_years_and_over',
'SEX_AND_AGE_Total_population_21_years_and_over',
'SEX_AND_AGE_Total_population_62_years_and_over',
'SEX_AND_AGE_Total_population_65_years_and_over',
'SEX_AND_AGE_Total_population_18_years_and_over.1',
'SEX_AND_AGE_Total_population_18_years_and_over_Male',
'SEX_AND_AGE_Total_population_18_years_and_over_Female',
'SEX_AND_AGE_Total_population_65_years_and_over.1',
'SEX_AND_AGE_Total_population_65_years_and_over_Male',
'SEX_AND_AGE_Total_population_65_years_and_over_Female']
df = df.drop(columns=drop_columns)

In [24]:
drop_columns = ['RACE_Total_population',
'RACE_Total_population_One_race',
'RACE_Total_population_Two_or_more_races',
'RACE_Total_population_One_race.1',
'RACE_Total_population_One_race_White',
'RACE_Total_population_One_race_Black_or_African_American',
'RACE_Total_population_One_race_American_Indian_and_Alaska_Native',
'RACE_Total_population_One_race_American_Indian_and_Alaska_Native_Cherokee_tribal_grouping',
'RACE_Total_population_One_race_American_Indian_and_Alaska_Native_Chippewa_tribal_grouping',
'RACE_Total_population_One_race_American_Indian_and_Alaska_Native_Navajo_tribal_grouping',
'RACE_Total_population_One_race_American_Indian_and_Alaska_Native_Sioux_tribal_grouping',
'RACE_Total_population_One_race_Asian',
'RACE_Total_population_One_race_Asian_Asian_Indian',
'RACE_Total_population_One_race_Asian_Chinese',
'RACE_Total_population_One_race_Asian_Filipino',
'RACE_Total_population_One_race_Asian_Japanese',
'RACE_Total_population_One_race_Asian_Korean',
'RACE_Total_population_One_race_Asian_Vietnamese',
'RACE_Total_population_One_race_Asian_Other_Asian',
'RACE_Total_population_One_race_Native_Hawaiian_and_Other_Pacific_Islander',
'RACE_Total_population_One_race_Native_Hawaiian_and_Other_Pacific_Islander_Native_Hawaiian',
'RACE_Total_population_One_race_Native_Hawaiian_and_Other_Pacific_Islander_Guamanian_or_Chamorro',
'RACE_Total_population_One_race_Native_Hawaiian_and_Other_Pacific_Islander_Samoan',
'RACE_Total_population_One_race_Native_Hawaiian_and_Other_Pacific_Islander_Other_Pacific_Islander',
'RACE_Total_population_One_race_Some_other_race',
'RACE_Total_population_Two_or_more_races.1',
'RACE_Total_population_Two_or_more_races_White_and_Black_or_African_American',
'RACE_Total_population_Two_or_more_races_White_and_American_Indian_and_Alaska_Native',
'RACE_Total_population_Two_or_more_races_White_and_Asian',
'RACE_Total_population_Two_or_more_races_Black_or_African_American_and_American_Indian_and_Alaska_Native']
df = df.drop(columns=drop_columns)

In [25]:
drop_columns = ['Percent_RACE_Total_population',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_White',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Black_or_African_American',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_American_Indian_and_Alaska_Native',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Asian',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Native_Hawaiian_and_Other_Pacific_Islander',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Some_other_race']
df = df.drop(columns=drop_columns)

In [26]:
drop_columns = ['HISPANIC_OR_LATINO_AND_RACE_Total_population',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)_Mexican',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)_Puerto_Rican',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)_Cuban',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)_Other_Hispanic_or_Latino',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_White_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Black_or_African_American_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_American_Indian_and_Alaska_Native_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Asian_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Native_Hawaiian_and_Other_Pacific_Islander_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Some_other_race_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Two_or_more_races',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Two_or_more_races_Two_races_including_Some_other_race',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Two_or_more_races_Two_races_excluding_Some_other_race_and_Three_or_more_races']
df = df.drop(columns=drop_columns)


In [27]:
drop_columns= ['Below_poverty_level_Population_for_whom_poverty_status_is_determined',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_Under_5_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_5_to_17_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_Related_children_of_householder_under_18_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years_18_to_34_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years_35_to_64_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_60_years_and_over',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_65_years_and_over',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_SEX_Male',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_SEX_Female',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_White_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Black_or_African_American_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_American_Indian_and_Alaska_Native_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Asian_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Native_Hawaiian_and_Other_Pacific_Islander_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Some_other_race_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Two_or_more_races',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Hispanic_or_Latino_origin_(of_any_race)',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_White_alone,_not_Hispanic_or_Latino',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Less_than_high_school_graduate',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_High_school_graduate_(includes_equivalency)',
"Below_poverty_level_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Some_college,_associate's_degree",
"Below_poverty_level_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Bachelor's_degree_or_higher",
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed_Male',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed_Female',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed_Male',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed_Female',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Worked_full-time,_year-round_in_the_past_12_months',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Worked_part-time_or_part-year_in_the_past_12_months',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Did_not_work',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Male',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Female',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_15_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_16_to_17_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_18_to_24_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_25_to_34_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_35_to_44_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_45_to_54_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_55_to_64_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_65_to_74_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_75_years_and_over',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Worked_full-time,_year-round_in_the_past_12_months',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Worked_less_than_full-time,_year-round_in_the_past_12_months',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Did_not_work',
              'Unnamed:_0',
"Total_Population_for_whom_poverty_status_is_determined",
"Total_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years",
"Total_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_Under_5_years",
"Total_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_5_to_17_years",
"Total_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_Related_children_of_householder_under_18_years",
"Total_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years",
"Total_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years_18_to_34_years",
"Total_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years_35_to_64_years",
"Total_Population_for_whom_poverty_status_is_determined_AGE_60_years_and_over",
"Total_Population_for_whom_poverty_status_is_determined_AGE_65_years_and_over",
"Total_Population_for_whom_poverty_status_is_determined_SEX_Male",
"Total_Population_for_whom_poverty_status_is_determined_SEX_Female",
"Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_White_alone",
"Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Black_or_African_American_alone",
"Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_American_Indian_and_Alaska_Native_alone",
"Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Asian_alone",
"Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Native_Hawaiian_and_Other_Pacific_Islander_alone",
"Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Some_other_race_alone",
"Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Two_or_more_races",
"Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Hispanic_or_Latino_origin_(of_any_race)",
"Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_White_alone,_not_Hispanic_or_Latino",
"Total_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over",
"Total_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Less_than_high_school_graduate",
"Total_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_High_school_graduate_(includes_equivalency)",
"Total_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Some_college,_associate's_degree",
"Total_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Bachelor's_degree_or_higher",
"Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over",
"Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed",
"Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed_Male",
"Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed_Female",
"Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed",
"Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed_Male",
"Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed_Female",
"Total_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over",
"Total_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Worked_full-time,_year-round_in_the_past_12_months",
"Total_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Worked_part-time_or_part-year_in_the_past_12_months",
"Total_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Did_not_work",
"Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_50_percent_of_poverty_level",
"Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_125_percent_of_poverty_level",
"Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_150_percent_of_poverty_level",
"Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_185_percent_of_poverty_level",
"Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_200_percent_of_poverty_level",
"Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_300_percent_of_poverty_level",
"Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_400_percent_of_poverty_level",
"Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_500_percent_of_poverty_level",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Male",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Female",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_15_years",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_16_to_17_years",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_18_to_24_years",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_25_to_34_years",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_35_to_44_years",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_45_to_54_years",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_55_to_64_years",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_65_to_74_years",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_75_years_and_over",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Worked_full-time,_year-round_in_the_past_12_months",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Worked_less_than_full-time,_year-round_in_the_past_12_months",
"Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Did_not_work"]
df = df.drop(columns=drop_columns)


In [28]:
dummy = pd.get_dummies(df.DRG_label)

for i in dummy.columns:
    df[i] = dummy[i]
df = df.drop(columns=['DRG_Definition', 'DRG_label'])
for i in df.columns:
    print(i)

SEX_AND_AGE_Total_population
Percent_SEX_AND_AGE_Total_population_Male
Percent_SEX_AND_AGE_Total_population_Female
SEX_AND_AGE_Total_population_Sex_ratio_(males_per_100_females)
Percent_SEX_AND_AGE_Total_population_Under_5_years
Percent_SEX_AND_AGE_Total_population_5_to_9_years
Percent_SEX_AND_AGE_Total_population_10_to_14_years
Percent_SEX_AND_AGE_Total_population_15_to_19_years
Percent_SEX_AND_AGE_Total_population_20_to_24_years
Percent_SEX_AND_AGE_Total_population_25_to_34_years
Percent_SEX_AND_AGE_Total_population_35_to_44_years
Percent_SEX_AND_AGE_Total_population_45_to_54_years
Percent_SEX_AND_AGE_Total_population_55_to_59_years
Percent_SEX_AND_AGE_Total_population_60_to_64_years
Percent_SEX_AND_AGE_Total_population_65_to_74_years
Percent_SEX_AND_AGE_Total_population_75_to_84_years
Percent_SEX_AND_AGE_Total_population_85_years_and_over
SEX_AND_AGE_Total_population_Median_age_(years)
Percent_SEX_AND_AGE_Total_population_Under_18_years
Percent_SEX_AND_AGE_Total_population_16_years_

In [29]:
for i in df.columns:
    if df[i].dtype == object:
        print(i)

County
State
CountyState
county_name
Provider_Name
Provider_Street_Address
Provider_City
Provider_State
Hospital_Referral_Region_Description
DRG_id


In [30]:
for i in df.columns:
    print(f"'{i}',")

'SEX_AND_AGE_Total_population',
'Percent_SEX_AND_AGE_Total_population_Male',
'Percent_SEX_AND_AGE_Total_population_Female',
'SEX_AND_AGE_Total_population_Sex_ratio_(males_per_100_females)',
'Percent_SEX_AND_AGE_Total_population_Under_5_years',
'Percent_SEX_AND_AGE_Total_population_5_to_9_years',
'Percent_SEX_AND_AGE_Total_population_10_to_14_years',
'Percent_SEX_AND_AGE_Total_population_15_to_19_years',
'Percent_SEX_AND_AGE_Total_population_20_to_24_years',
'Percent_SEX_AND_AGE_Total_population_25_to_34_years',
'Percent_SEX_AND_AGE_Total_population_35_to_44_years',
'Percent_SEX_AND_AGE_Total_population_45_to_54_years',
'Percent_SEX_AND_AGE_Total_population_55_to_59_years',
'Percent_SEX_AND_AGE_Total_population_60_to_64_years',
'Percent_SEX_AND_AGE_Total_population_65_to_74_years',
'Percent_SEX_AND_AGE_Total_population_75_to_84_years',
'Percent_SEX_AND_AGE_Total_population_85_years_and_over',
'SEX_AND_AGE_Total_population_Median_age_(years)',
'Percent_SEX_AND_AGE_Total_population_Under_

In [31]:
select = ['SEX_AND_AGE_Total_population',"Households_Total",
    "Households_Less_than_$10,000",
    "Households_$10,000_to_$14,999",
    "Households_$15,000_to_$24,999",
    "Households_$25,000_to_$34,999",
    "Households_$35,000_to_$49,999",
    "Households_$50,000_to_$74,999",
    "Households_$100,000_to_$149,999",
    "Households_$150,000_to_$199,999",
    "Households_$200,000_or_more",
    "Households_Median_income_(dollars)",
    "Households_Mean_income_(dollars)",
    "Households_PERCENT_IMPUTED_Household_income_in_the_past_12_months",
    "Families_Total",
    "Families_Less_than_$10,000",
    "Families_$10,000_to_$14,999",
    "Families_$15,000_to_$24,999",
    "Families_$25,000_to_$34,999",
    "Families_$35,000_to_$49,999",
    "Families_$50,000_to_$74,999",
    "Families_$75,000_to_$99,999",
    "Families_$100,000_to_$149,999",
    "Families_$150,000_to_$199,999",
    "Families_$200,000_or_more",
    "Families_Median_income_(dollars)",
    "Families_Mean_income_(dollars)",
    "Families_PERCENT_IMPUTED_Family_income_in_the_past_12_months",
    "Married-couple_families_Median_income_(dollars)",
    "Households_$75,000_to_$99,999",
    "Nonfamily_households_Median_income_(dollars)",
"Nonfamily_households_PERCENT_IMPUTED_Nonfamily_income_in_the_past_12_months",
'ACUTE MYOCARDIAL INFARCTION, DISCHARGED ALIVE',
'ALCOHOL/DRUG ABUSE OR DEPENDENCE',
'ATHEROSCLEROSIS',
'BACK & NECK PROC EXC SPINAL FUSION',
'BRONCHITIS & ASTHMA',
'CARDIAC ARRHYTHMIA & CONDUCTION DISORDERS',
'CELLULITIS',
'CERVICAL SPINAL FUSION',
'CHEST PAIN',
'CHRONIC OBSTRUCTIVE PULMONARY DISEASE',
'CIRCULATORY DISORDERS EXCEPT AMI,',
'CRANIAL & PERIPHERAL NERVE DISORDERS',
'DEGENERATIVE NERVOUS SYSTEM DISORDERS',
'DIABETES',
'DISORDERS OF PANCREAS EXCEPT MALIGNANCY',
'DYSEQUILIBRIUM',
'ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS',
'EXTRACRANIAL PROCEDURES',
'FRACTURES OF HIP & PELVIS',
'FX, SPRN, STRN & DISL EXCEPT FEMUR, HIP, PELVIS & THIGH',
'G.I. HEMORRHAGE',
'G.I. OBSTRUCTION',
'HEART FAILURE & SHOCK',
'HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT',
'HYPERTENSION',
'INFECTIOUS & PARASITIC DISEASES',
'INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION',
'KIDNEY & URINARY TRACT INFECTIONS',
'LAPAROSCOPIC CHOLECYSTECTOMY',
'MAJOR CARDIOVASC PROCEDURES',
'MAJOR GASTROINTESTINAL DISORDERS & PERITONEAL INFECTIONS',
'MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY',
'MAJOR SMALL & LARGE BOWEL PROCEDURES',
'MEDICAL BACK PROBLEMS',
'MISC DISORDERS OF NUTRITION,METABOLISM,FLUIDS/ELECTROLYTES',
'OTHER CIRCULATORY SYSTEM DIAGNOSES',
'OTHER DIGESTIVE SYSTEM DIAGNOSES',
'OTHER KIDNEY & URINARY TRACT DIAGNOSES',
'OTHER VASCULAR PROCEDURES',
'PERC CARDIOVASC PROC',
'PERIPHERAL VASCULAR DISORDERS',
'PERMANENT CARDIAC PACEMAKER IMPLANT',
'POISONING & TOXIC EFFECTS OF DRUGS',
'PSYCHOSES',
'PULMONARY EDEMA & RESPIRATORY FAILURE',
'PULMONARY EMBOLISM',
'RED BLOOD CELL DISORDERS',
'RENAL FAILURE',
'RESPIRATORY INFECTIONS & INFLAMMATIONS',
'RESPIRATORY SYSTEM DIAGNOSIS',
'SEIZURES',
'SEPTICEMIA OR SEVERE SEPSIS',
'SIGNS & SYMPTOMS',
'SIMPLE PNEUMONIA & PLEURISY',
'SPINAL FUSION EXCEPT CERVICAL',
'SYNCOPE & COLLAPSE',
"ratio_to_max_payment"]


In [32]:
count = 1
for i in select:
    try:
        print(df[i].dtype, count)
    except:
        print(i, count)
    count += 1 

int64 1
float64 2
float64 3
float64 4
float64 5
float64 6
float64 7
float64 8
float64 9
float64 10
float64 11
float64 12
float64 13
float64 14
float64 15
float64 16
float64 17
float64 18
float64 19
float64 20
float64 21
float64 22
float64 23
float64 24
float64 25
float64 26
float64 27
float64 28
float64 29
float64 30
float64 31
float64 32
uint8 33
uint8 34
uint8 35
uint8 36
uint8 37
uint8 38
uint8 39
uint8 40
uint8 41
uint8 42
uint8 43
uint8 44
uint8 45
uint8 46
uint8 47
uint8 48
uint8 49
uint8 50
uint8 51
uint8 52
uint8 53
uint8 54
uint8 55
uint8 56
uint8 57
uint8 58
uint8 59
uint8 60
uint8 61
uint8 62
uint8 63
uint8 64
uint8 65
uint8 66
uint8 67
uint8 68
uint8 69
uint8 70
uint8 71
uint8 72
uint8 73
uint8 74
uint8 75
uint8 76
uint8 77
uint8 78
uint8 79
uint8 80
uint8 81
uint8 82
uint8 83
uint8 84
uint8 85
uint8 86
uint8 87
uint8 88
float64 89


In [33]:
data.corr()['ratio_to_max_payment']


Unnamed:_0                   0.002377
Provider_Id                 -0.073806
Provider_Zip_Code            0.023456
_Total_Discharges_           0.011171
_Average_Covered_Charges_    0.065269
_Average_Total_Payments_     0.135792
Average_Medicare_Payments    0.126276
without_ccmcc               -0.017264
with_mcc                    -0.108631
with_cc                      0.169525
with_ccmcc                  -0.158958
ratio_to_max_payment         1.000000
ratio_to_max_discharge       0.075101
Name: ratio_to_max_payment, dtype: float64

In [53]:
for i in df.columns:
    print(f'"{i}",')
    
    "without_ccmcc",
"with_mcc",
"with_cc",
"with_ccmcc",

"Unnamed:_0",
"county_name",
"DRG_Definition",
"Provider_Id",
"Provider_Name",
"Provider_Street_Address",
"Provider_City",
"Provider_State",
"Provider_Zip_Code",
"Hospital_Referral_Region_Description",
"_Total_Discharges_",
"_Average_Covered_Charges_",
"_Average_Total_Payments_",
"Average_Medicare_Payments",
"DRG_id",
"DRG_label",
"without_ccmcc",
"with_mcc",
"with_cc",
"with_ccmcc",
"ratio_to_max_payment",
"ratio_to_max_discharge",
"CountyState",
"SEX_AND_AGE_Total_population",
"Percent_SEX_AND_AGE_Total_population",
"SEX_AND_AGE_Total_population_Male",
"Percent_SEX_AND_AGE_Total_population_Male",
"SEX_AND_AGE_Total_population_Female",
"Percent_SEX_AND_AGE_Total_population_Female",
"SEX_AND_AGE_Total_population_Sex_ratio_(males_per_100_females)",
"SEX_AND_AGE_Total_population_Under_5_years",
"Percent_SEX_AND_AGE_Total_population_Under_5_years",
"SEX_AND_AGE_Total_population_5_to_9_years",
"Percent_SEX_AND_AGE_Total_population_5_to_9_years",
"SEX_AND_AGE_Total_population_10_to_14_years

In [54]:
pd.merge(data, acs_data, how='left', on=['CountyState'])

,Unnamed:_0,county_name,DRG_Definition,Provider_Id,Provider_Name,Provider_Street_Address,Provider_City,Provider_State,Provider_Zip_Code,Hospital_Referral_Region_Description,...,"Families_$200,000_or_more",Families_Median_income_(dollars),Families_Mean_income_(dollars),Families_PERCENT_IMPUTED_Family_income_in_the_past_12_months,Married-couple_families_Median_income_(dollars),"Households_$75,000_to_$99,999",Nonfamily_households_Median_income_(dollars),Nonfamily_households_PERCENT_IMPUTED_Nonfamily_income_in_the_past_12_months,County,State
0,0,Hampden,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,220024,HOLYOKE MEDICAL CENTER,575 BEECH STREET,HOLYOKE,MA,1040,MA - Springfield,...,5.2,60491.0,77563.0,29.3,83305.0,11.8,29546.0,25.2,Hampden,MA
1,1,Hampden,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,220024,HOLYOKE MEDICAL CENTER,575 BEECH STREET,HOLYOKE,MA,1040,MA - Springfield,...,5.2,60491.0,77563.0,29.3,83305.0,11.8,29546.0,25.2,Hampden,MA
2,2,Hampden,065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,220024,HOLYOKE MEDICAL CENTER,575 BEECH STREET,HOLYOKE,MA,1040,MA - Springfield,...,5.2,60491.0,77563.0,29.3,83305.0,11.8,29546.0,25.2,Hampden,MA
3,3,Hampden,069 - TRANSIENT ISCHEMIA,220024,HOLYOKE MEDICAL CENTER,575 BEECH STREET,HOLYOKE,MA,1040,MA - Springfield,...,5.2,60491.0,77563.0,29.3,83305.0,11.8,29546.0,25.2,Hampden,MA
4,4,Hampden,101 - SEIZURES W/O MCC,220024,HOLYOKE MEDICAL CENTER,575 BEECH STREET,HOLYOKE,MA,1040,MA - Springfield,...,5.2,60491.0,77563.0,29.3,83305.0,11.8,29546.0,25.2,Hampden,MA
5,5,Hampden,176 - PULMONARY EMBOLISM W/O MCC,220024,HOLYOKE MEDICAL CENTER,575 BEECH STREET,HOLYOKE,MA,1040,MA - Springfield,...,5.2,60491.0,77563.0,29.3,83305.0,11.8,29546.0,25.2,Hampden,MA
6,6,Hampden,177 - RESPIRATORY INFECTIONS & INFLAMMATIONS W...,220024,HOLYOKE MEDICAL CENTER,575 BEECH STREET,HOLYOKE,MA,1040,MA - Springfield,...,5.2,60491.0,77563.0,29.3,83305.0,11.8,29546.0,25.2,Hampden,MA
7,7,Hampden,178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC,220024,HOLYOKE MEDICAL CENTER,575 BEECH STREET,HOLYOKE,MA,1040,MA - Springfield,...,5.2,60491.0,77563.0,29.3,83305.0,11.8,29546.0,25.2,Hampden,MA
8,8,Hampden,189 - PULMONARY EDEMA & RESPIRATORY FAILURE,220024,HOLYOKE MEDICAL CENTER,575 BEECH STREET,HOLYOKE,MA,1040,MA - Springfield,...,5.2,60491.0,77563.0,29.3,83305.0,11.8,29546.0,25.2,Hampden,MA
9,9,Hampden,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,220024,HOLYOKE MEDICAL CENTER,575 BEECH STREET,HOLYOKE,MA,1040,MA - Springfield,...,5.2,60491.0,77563.0,29.3,83305.0,11.8,29546.0,25.2,Hampden,MA


In [66]:
multiple_county = [i for i in acs_data.CountyState.value_counts()[acs_data.CountyState.value_counts()>1].index]

In [69]:
acs_data.loc[acs_data.CountyState.apply(lambda x: x in multiple_county)]

,SEX_AND_AGE_Total_population,Percent_SEX_AND_AGE_Total_population,SEX_AND_AGE_Total_population_Male,Percent_SEX_AND_AGE_Total_population_Male,SEX_AND_AGE_Total_population_Female,Percent_SEX_AND_AGE_Total_population_Female,SEX_AND_AGE_Total_population_Sex_ratio_(males_per_100_females),SEX_AND_AGE_Total_population_Under_5_years,Percent_SEX_AND_AGE_Total_population_Under_5_years,SEX_AND_AGE_Total_population_5_to_9_years,...,Families_Median_income_(dollars),Families_Mean_income_(dollars),Families_PERCENT_IMPUTED_Family_income_in_the_past_12_months,Married-couple_families_Median_income_(dollars),"Households_$75,000_to_$99,999",Nonfamily_households_Median_income_(dollars),Nonfamily_households_PERCENT_IMPUTED_Nonfamily_income_in_the_past_12_months,County,State,CountyState
1149,619796,619796,291377,47.0,328419,53.0,88.7,40779,6.6,36018,...,47070.0,66508.0,39.8,74669.0,8.1,28926.0,30.1,Baltimore,MD,baltimoremd
1151,828637,828637,392930,47.4,435707,52.6,90.2,49096,5.9,51011,...,78914.0,98748.0,31.4,97106.0,13.1,40617.0,32.3,Baltimore,MD,baltimoremd
1530,999539,999539,474110,47.4,525429,52.6,90.2,58328,5.8,59937,...,73585.0,98113.0,29.9,91552.0,12.7,34367.0,25.7,St. Louis,MO,st. louismo
1570,314867,314867,152224,48.3,162643,51.7,93.6,20582,6.5,16509,...,42698.0,59055.0,33.4,65906.0,8.9,25101.0,26.4,St. Louis,MO,st. louismo
2831,8873,8873,4978,56.1,3895,43.9,127.8,308,3.5,360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Richmond,VA,richmondva
2838,56277,56277,27631,49.1,28646,50.9,96.5,2630,4.7,3309,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Franklin,VA,franklinva
2851,93419,93419,44811,48.0,48608,52.0,92.2,4491,4.8,5094,...,71974.0,88908.0,28.4,79864.0,14.9,29744.0,21.3,Roanoke,VA,roanokeva
2882,220892,220892,104853,47.5,116039,52.5,90.4,13393,6.1,11505,...,48553.0,76754.0,34.0,82812.0,8.5,30017.0,34.5,Richmond,VA,richmondva
2934,23580,23580,11621,49.3,11959,50.7,97.2,1662,7.0,1328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fairfax,VA,fairfaxva
2937,99572,99572,47736,47.9,51836,52.1,92.1,6854,6.9,6421,...,52982.0,62466.0,31.5,63706.0,8.2,26235.0,26.9,Roanoke,VA,roanokeva
